In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import requests
import json
from datetime import datetime

### 22.12.02 记录：

### 数据说明：

- 最新的原始数据：`2021_q2_new.csv`

- 最新的有效车位id： `1202_valid.csv`

- 最新的处理过后的数据集：`1201_result.csv`

--------------------------------------------------------

### 主要的处理：

- 关于valid_id: 检验除了M(明确标注active)外，其他标识的meter是否有transaction记录。

- 结论：有，但很少。只有四个标注了U的meter有交易记录（共计261条），现已经将这四个meter的加到最新的有效车位数据集：`1202_valid.csv`.

---------------------------------------------------------


----------------------------------------

1. 读取数据

- 读取整合过的数据（包含street_block）

- 更新过的valid_id，仅‘M’.

In [28]:
path1 = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\1201_result.csv'
result = pd.read_csv(path1)

In [29]:
result

,Unnamed: 0,post_id,session_start_dt,session_end_dt,interval
0,0,102-02990,2021-04-01 09:39:52,2021-04-01 11:39:52,7200
1,1,102-02990,2021-04-01 12:11:01,2021-04-01 12:48:31,2250
2,2,102-02990,2021-04-01 13:26:19,2021-04-01 13:56:19,1800
3,3,102-02990,2021-04-01 14:11:03,2021-04-01 15:06:19,3316
4,4,102-02990,2021-04-01 15:49:00,2021-04-01 16:09:00,1200
...,...,...,...,...,...
2878229,2878229,920-00350,2021-06-25 14:07:06,2021-06-25 14:10:48,222
2878230,2878230,920-00350,2021-06-25 14:10:53,2021-06-25 14:15:30,277
2878231,2878231,920-00350,2021-06-29 13:04:54,2021-06-29 15:37:13,9139
2878232,2878232,920-00350,2021-06-29 15:43:02,2021-06-29 16:38:26,3324


- valid_id:仅含M.

In [4]:
path = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\M_valid_meter.csv'

valid = pd.read_csv(path)

In [5]:
valid

,post_id
0,211-02380
1,596-00410
2,133-17960
3,650-33160
4,831-04430
...,...
15885,692-01500
15886,612-24310
15887,581-24440
15888,692-01220


-------------------------------
2.判断`result`中的id是否是‘M’

`result_T` 是valid id.

In [30]:
result['valid'] = result['post_id'].isin(valid['post_id'])

result_T = result.loc[result1['valid']==True]

result_T

,Unnamed: 0,post_id,session_start_dt,session_end_dt,interval,valid
0,0,102-02990,2021-04-01 09:39:52,2021-04-01 11:39:52,7200,True
1,1,102-02990,2021-04-01 12:11:01,2021-04-01 12:48:31,2250,True
2,2,102-02990,2021-04-01 13:26:19,2021-04-01 13:56:19,1800,True
3,3,102-02990,2021-04-01 14:11:03,2021-04-01 15:06:19,3316,True
4,4,102-02990,2021-04-01 15:49:00,2021-04-01 16:09:00,1200,True
...,...,...,...,...,...,...
2878229,2878229,920-00350,2021-06-25 14:07:06,2021-06-25 14:10:48,222,True
2878230,2878230,920-00350,2021-06-25 14:10:53,2021-06-25 14:15:30,277,True
2878231,2878231,920-00350,2021-06-29 13:04:54,2021-06-29 15:37:13,9139,True
2878232,2878232,920-00350,2021-06-29 15:43:02,2021-06-29 16:38:26,3324,True


去掉不需要的列：

In [53]:
res = result_T.loc[:,['post_id','session_start_dt','session_end_dt','interval']]

In [55]:
res.sort_values('interval')

,post_id,session_start_dt,session_end_dt,interval
997696,380-03030,2021-05-22 09:00:00,2021-05-22 09:00:01,1
2178077,600-17100,2021-04-26 17:59:59,2021-04-26 18:00:00,1
976818,372-06300,2021-06-05 17:59:59,2021-06-05 18:00:00,1
1934084,568-52030,2021-06-01 17:59:59,2021-06-01 18:00:00,1
2857540,855-00500,2021-06-04 09:00:00,2021-06-04 09:00:01,1
...,...,...,...,...
2796499,825-07110,2021-05-18 09:00:00,2021-05-18 18:00:00,32400
633328,331-04530,2021-06-03 09:00:00,2021-06-03 18:00:00,32400
1976968,577-04280,2021-04-26 09:00:00,2021-04-26 18:00:00,32400
1768457,552-06420,2021-05-07 09:00:00,2021-05-07 18:00:00,32400


In [61]:
path1 = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\1202_res.csv'
res.to_csv(path1)

有261条交易的`post_id`不是`M`类型。

- 出现在交易记录中但是id的标识不是M的数据的post_id:

In [31]:
result_F = result.loc[result1['valid']==False]

result_F['post_id'].unique()

array(['470-01210', '470-01230', '470-01250', '470-01270'], dtype=object)

In [32]:
result_F

,Unnamed: 0,post_id,session_start_dt,session_end_dt,interval,valid
1445576,1445576,470-01210,2021-04-02 12:13:00,2021-04-02 13:43:00,5400,False
1445577,1445577,470-01210,2021-04-02 17:16:05,2021-04-02 18:00:00,2635,False
1445578,1445578,470-01210,2021-04-10 14:47:00,2021-04-10 15:47:00,3600,False
1445579,1445579,470-01210,2021-04-13 12:01:00,2021-04-13 14:01:00,7200,False
1445580,1445580,470-01210,2021-04-14 10:19:00,2021-04-14 12:19:00,7200,False
...,...,...,...,...,...,...
1445832,1445832,470-01270,2021-06-19 14:34:31,2021-06-19 16:34:31,7200,False
1445833,1445833,470-01270,2021-06-21 09:13:00,2021-06-21 11:13:00,7200,False
1445834,1445834,470-01270,2021-06-25 13:20:00,2021-06-25 14:20:00,3600,False
1445835,1445835,470-01270,2021-06-25 14:20:00,2021-06-25 15:00:00,2400,False


In [33]:
valid

,post_id
0,211-02380
1,596-00410
2,133-17960
3,650-33160
4,831-04430
...,...
15885,692-01500
15886,612-24310
15887,581-24440
15888,692-01220


- 将这些id整合到valid id里面：

In [40]:
v1 = pd.DataFrame({'post_id':['470-01210', '470-01230', '470-01250', '470-01270']})

In [41]:
v1

,post_id
0,470-01210
1,470-01230
2,470-01250
3,470-01270


In [44]:
v2 = valid.append(v1,ignore_index = True)

C:\Users\yyp\AppData\Local\Temp\ipykernel_20940\2143705216.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  v2 = valid.append(v1,ignore_index = True)


In [57]:
v2

,post_id
0,211-02380
1,596-00410
2,133-17960
3,650-33160
4,831-04430
...,...
15889,722-29080
15890,470-01210
15891,470-01230
15892,470-01250


In [62]:
path1 = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\1202_valid.csv'
v2.to_csv(path1)